In [1]:
# Задаем дату до которой теоритически будем прогнозировать
end_future_date = '2024-12-31 23:55:00+00:00'

In [2]:
import pandas as pd

# Читаем данные на которых будем обучать модель
df_Pl = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')


In [3]:
# Узнаем последнию известную дату. Нужна для создания промежутка значений будущих дат
end_know_date = df_Pl['time'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0] + 1


In [4]:
import numpy as np

# Создаем общий датасет с известными и будущими данными нужно для корректного общего нормирования данных

# Добавляем 5 минут чтобы последняя известная дата не попала в список для генерации будущих дат

end_know_date = pd.to_datetime(end_know_date)

# Добавление 5 минут
end_know_date = end_know_date + pd.Timedelta(minutes=5)

# Преобразование обратно в строку
end_know_date = end_know_date.strftime('%Y-%m-%d %H:%M:%S%z')

future_dates = pd.date_range(start=end_know_date, end=end_future_date, freq='5T')

future_df = pd.DataFrame({'time': future_dates, 'P_l': np.nan})

avg_pl = df_Pl['P_l'].mean()

# Заполняем Nan если они есть в датафрейме с известными данными
df_Pl['P_l'].fillna(avg_pl, inplace=True)

# Создаем общий датаферйм с известными и будущими датами для корректной нормировки данных относительно существующих и будущих дат
df_Pl_sacler = pd.concat([df_Pl, future_df], ignore_index=True)


In [5]:
# Функция нормировки дат относительно всех дат(известных и которых хотим прогнощировать)

def normalized_df(df, scaler):
# Преобразование времени в формат datetime
  df['time'] = pd.to_datetime(df['time'])
  # Создание новых столбцов
  df['year'] = df['time'].dt.year
  df['month'] = df['time'].dt.month
  df['day'] = df['time'].dt.day
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  # Удаление исходной колонки времени
  df = df.drop('time', axis=1)

  # Нормализация данных
  df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
  return df_normalized

# Обратное масштабирование данных времени
def restored_date(df_normalized, scaler):
  df_restored = pd.DataFrame(scaler.inverse_transform(df_normalized), columns=df_normalized.columns)
  df_restored['time'] = pd.to_datetime(df_restored[['year', 'month', 'day', 'hour', 'minute']])
  df_restored = df_restored.drop(['year', 'month', 'day', 'hour', 'minute'], axis=1)
  return df_restored

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Нормализация всех данных существующих и будущих
scaler = MinMaxScaler(feature_range=(0, 1))

df_Pl_normalized_all_dates = normalized_df(df_Pl_sacler, scaler)


In [7]:
# Данные для обучения отнормированные относительно всего промежутка(известных дат и дат для прогноза)

df_Pl_normalized_initial_dates = df_Pl_normalized_all_dates.loc[: end_know_date_index-1]


In [8]:
df_Pl_normalized_future_dates = df_Pl_normalized_all_dates.loc[end_know_date_index :]


In [9]:
# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values


In [10]:

# gреобразование в 5 мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)


In [11]:
from sklearn.model_selection import train_test_split

# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values

# Преобразование в 5-мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_initial_train, y_initial, test_size=0.01, random_state=42)


In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, concatenate
from tensorflow.keras.models import Model

print(X_train.shape)
# Создание входов
input_cnn = Input(shape=(1, 5), name='input_cnn')
input_lstm = Input(shape=(1, 5), name='input_lstm')
print(input_cnn)
# Сверточный слой для признака P_l
conv1 = Conv1D(filters=32, kernel_size=1, activation='relu')(input_cnn)  # изменено kernel_size на 1
pool1 = MaxPooling1D(pool_size=1)(conv1)
flat1 = Flatten()(pool1)

# LSTM слой для времени
lstm = LSTM(units=50, activation='relu')(input_lstm)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([flat1, lstm])

dense = Dense(units=50, activation='relu')(merged)

output = Dense(units=1, activation='linear')(dense)

model = Model(inputs=[input_cnn, input_lstm], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Обучение модели
epochs = 50
batch_size = 8
model.fit([X_train, X_train], y_train, epochs=epochs, batch_size=batch_size)


(755262, 1, 5)
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 5), dtype=tf.float32, name='input_cnn'), name='input_cnn', description="created by layer 'input_cnn'")
Epoch 1/50
94408/94408 [==============================] - 79s 831us/step - loss: 0.0018
Epoch 2/50
94408/94408 [==============================] - 79s 832us/step - loss: 0.0014
Epoch 3/50
94408/94408 [==============================] - 79s 837us/step - loss: 0.0014
Epoch 4/50
94408/94408 [==============================] - 79s 841us/step - loss: 0.0013
Epoch 5/50
94408/94408 [==============================] - 78s 831us/step - loss: 0.0013
Epoch 6/50
94408/94408 [==============================] - 78s 830us/step - loss: 0.0013
Epoch 7/50
44933/94408 [=============>................] - ETA: 41s - loss: 0.0013

In [ ]:
model.save(f'/Users/nikitasavvin/Desktop/Учеба/CNN-LSTM/lstm_clean/models/model_epochs_{epochs}_batch_size_{batch_size}.h5')


In [ ]:
from tensorflow.keras.models import load_model

model = load_model(f'/Users/nikitasavvin/Desktop/Учеба/CNN-LSTM/lstm_clean/models/model_epochs_{epochs}_batch_size_{batch_size}.h5')


In [ ]:
import pandas as pd

def get_indices_between_dates(df_Pl_sacler, start_date, end_date):
    """
    Получает DataFrame, начальную и конечную даты, возвращает список индексов,
    соответствующих датам в заданном диапазоне.
    Нужна для получения индексов времени которые мы хотим спрогнозировать
    Parameters:
    - df_Pl_sacler: pandas.DataFrame
        DataFrame с колонкой 'time', содержащей даты в формате 'yyyy-mm-dd hh:mm:ss+00:00'.
    - start_date: str
        Начальная дата в формате 'yyyy-mm-dd'.
    - end_date: str
        Конечная дата в формате 'yyyy-mm-dd'.

    Returns:
    - list
        Список индексов, соответствующих датам в заданном диапазоне.
    """
    # Преобразование колонки 'time' в формат datetime
    df_Pl_sacler['time'] = pd.to_datetime(df_Pl_sacler['time'])

    # Выборка индексов для дат в заданном диапазоне
    indices = df_Pl_sacler.loc[(df_Pl_sacler['time'] >= start_date) & (df_Pl_sacler['time'] <= end_date)].index

    return indices



In [ ]:
'''
Блок получения нормированных дат для подстановки в модель
'''

# Даты на которые хотим сделать прогноз
start_date = '2023-09-01'
end_date = '2023-09-06'

df_Pl_all_dates = df_Pl_sacler.copy()

# выбираем индексы для которых хотим сделать прогноз из общего датасета(с бущими и настоящими датами)
indexes = get_indices_between_dates(df_Pl_all_dates, start_date, end_date)

# Нормализуем датасет чтобы получить нормализованные даты чтобы корректно поместить их в модель для предсказания
df_Pl_all_dates_normilize = normalized_df(df_Pl_all_dates, scaler)

# Получаем датафрейм с нормированными датами на промежуток прогноза
df_Pl_sacler_for_predict = df_Pl_all_dates_normilize.loc[indexes]

X_predict = df_Pl_sacler_for_predict[['year', 'month', 'day', 'hour', 'minute']].values

X_predict = X_predict.reshape(-1, 1, 5)

In [ ]:
# Предсказания
X_predict_scaled = X_predict
X_predict_cnn = X_predict
X_predict_lstm = X_predict

predictions = model.predict([X_predict_cnn, X_predict_lstm])

# Обратное масштабирование предсказанных значений

In [ ]:
df_predict = df_Pl_sacler_for_predict.copy()
df_predict['P_l'] = predictions.flatten()

df_predict = restored_date(df_predict, scaler)

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')

df_comporative = df.loc[indexes]

In [ ]:
df_comporative['time'] = df_comporative['time'].astype(df_predict['time'].dtype)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(df_predict['time'], df_predict['P_l'], linewidth=1)
plt.plot(df_comporative['time'], df_comporative['P_l'], linewidth=1, color='y')
plt.title(label="Predict")
plt.ylabel("P_l Value")
plt.xlabel("Time")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


def calculate_metrics(df_orig, df_test):
    avg_pl = df_Pl['P_l'].mean()

# Заполняем Nan если они есть в датафрейме с известными данными
    df_orig['P_l'].fillna(avg_pl, inplace=True)



    y_true = df_orig
    y_pred = df_test

    y_true_mean = y_true['P_l'].mean()
        # Расчет RMSE
    rmse = np.sqrt(mean_squared_error(y_true['P_l'].values , y_pred['P_l'].values ))

    # Расчет R^2
    ss_res = np.sum((y_true['P_l'].values  - y_pred['P_l'].values ) ** 2)
    ss_tot = np.sum((y_true['P_l'].values  - y_true_mean) ** 2)
    r2 = 1 - (ss_res / ss_tot)

    # Расчет MAE
    mae = mean_absolute_error(y_true['P_l'].values , y_pred['P_l'].values )

    # Расчет MAPE
    mape = np.mean(np.abs((y_true['P_l'].values - y_pred['P_l'].values) / y_true['P_l'].values)) * 100
    # Расчет WMAPE
    wmape = np.sum(np.abs(y_true['P_l'].values  - y_pred['P_l'].values )) / np.sum(np.abs(y_true['P_l'].values )) * 100

    return rmse, r2, mae, mape, wmape

In [ ]:
rmse, r2, mae, mape, wmape = calculate_metrics(df_comporative, df_predict)

In [ ]:
print(f'RMSE = {rmse}')
print(f'R-squared = {r2}')
print(f'MAE = {mae}')
print(f'MAPE = {mape}')
print(f'WMAPE = {wmape}')